In [ ]:
import pandas as pd
import googlemaps
import time

gmaps = googlemaps.Client(key='YOUR_API_KEY_HERE')

In [ ]:
# Load dataset
df = pd.read_csv('beer_reviews.csv')

# Get unique brewery names and save them to a CSV file
unique_breweries = df['brewery_name'].unique()
unique_breweries_df = pd.DataFrame(unique_breweries, columns=['brewery_name'])
unique_breweries_df.to_csv('unique_breweries.csv', index=False)

In [ ]:
# Read the unique breweries file
unique_breweries_df = pd.read_csv('unique_breweries.csv')

# Define the chunk size
chunk_size = 500

# Split the DataFrame into smaller chunks and save them as separate CSVs 
for i in range(0, len(unique_breweries_df), chunk_size):
    chunk = unique_breweries_df[i:i+chunk_size]
    chunk.to_csv(f'brewery_chunk_{i // chunk_size + 1}.csv', index=False)

In [ ]:
def get_lat_lng(brewery_name):
  try:
    result = gmaps.places(query=brewery_name) 

    if result['results']:  
      location = result['results'][0]['geometry']['location']
      return location['lat'], location['lng'] 
    else:
      return None, None  

  except Exception as e:
    print(f"Error processing {brewery_name}: {e}")
    return None, None 

In [ ]:
# Variables for API request management
requests_made = 0
max_requests_per_minute = 90 

def process_brewery_chunk(chunk_file):
  chunk_df = pd.read_csv(chunk_file)
  brewery_coords = {}

  for i, brewery in enumerate(chunk_df['brewery_name']):
    lat, lng = get_lat_lng(brewery)
    brewery_coords[brewery] = (lat, lng)
    print(f"Processed {i+1}/{len(chunk_df)}: {brewery} - Lat: {lat}, Lng: {lng}")

    # API Request Management
    global requests_made
    requests_made += 1

    if requests_made >= max_requests_per_minute:
      print("Reached the maximum number of requests per minute. Sleeping for 60 seconds.")
      time.sleep(60)
      requests_made = 0

  return brewery_coords

In [ ]:
def process_chunk():
    try:
        chunk_file_name = input("Enter the name of the chunk file to process: ") 

        brewery_coords = process_brewery_chunk(chunk_file_name) 

        coord_df = pd.DataFrame.from_dict(brewery_coords, orient='index', columns=['latitude', 'longitude'])
        coord_df.reset_index(inplace=True)
        coord_df.rename(columns={'index': 'brewery_name'}, inplace=True)

        output_filename = chunk_file_name.replace('brewery_chunk_', 'chunk_') + '_coordinates.csv'
        coord_df.to_csv(output_filename, index=False)

        print("Chunk processing complete! Coordinates saved.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to process chunks
process_chunk()